In [1]:
# 설치
!pip install --upgrade openai

import os
import re
import openai
import json
from google.colab import userdata

# OpenAI API 키 설정

openai.api_key = userdata.get('OPENAI_API_KEY')

# 하이브리드 입력 처리 함수

def rule_based_extraction(text):
    result = {"성별": None, "지역": None, "소득": None, "나이": None, "직업": None}
    if re.search(r"(남자|남성)", text): result["성별"] = "남성"
    elif re.search(r"(여자|여성)", text): result["성별"] = "여성"
    regions = ["서울", "부산", "대전", "대구", "광주", "인천", "울산", "세종"]
    for r in regions:
        if r in text: result["지역"] = r; break
    income_pattern = re.search(r"(\d+(?:천)?)\s*만원", text)
    if income_pattern:
        income_str = income_pattern.group(1)
        if "이하" in text: result["소득"] = f"{income_str}만원 이하"
        elif "이상" in text: result["소득"] = f"{income_str}만원 이상"
        else: result["소득"] = f"{income_str}만원"
    age_pattern = re.search(r"(\d+)\s*(살|세)", text)
    if age_pattern: result["나이"] = int(age_pattern.group(1))
    if re.search(r"(학생|대학생|고등학생|중학생)", text): result["직업"] = "학생"
    elif re.search(r"(직장인|회사원|사무직|개발자|프리랜서|자영업자)", text): result["직업"] = "직장인"
    else: result["직업"] = "무직/기타"
    return result

def gpt_fallback(text, existing_data):
    missing_items = {k: v for k, v in existing_data.items() if v is None}
    prompt = f"""
    사용자의 입력 텍스트는 다음과 같아: "{text}"
    현재까지 파악된 정보는 다음과 같아: {existing_data}
    사용자 텍스트를 참고해서 아직 파악되지 않은 '{list(missing_items.keys())}' 항목을 추론하고, 최종 결과를 완결된 JSON 객체로 출력해줘.
    다른 설명은 절대 추가하지 마.
    """
    response = openai.chat.completions.create(
        model="gpt-4o", response_format={"type": "json_object"},
        messages=[{"role": "user", "content": prompt}], temperature=0
    )
    message_content = response.choices[0].message.content
    try: return json.loads(message_content)
    except json.JSONDecodeError: print("GPT 응답이 JSON이 아닙니다:", message_content); return {**existing_data, **missing_items}

def extract_user_info(text):
    rule_result = rule_based_extraction(text)
    if None in rule_result.values(): final_result = gpt_fallback(text, rule_result)
    else: final_result = rule_result
    return final_result

In [2]:
# 추천 시스템 함수

def create_recommendation_prompt(user_info):
    system_prompt = f"""
    당신은 한국의 복지정책, 장학금, 지원사업의 상세 조건을 분석하는 AI 전문가입니다.
    현재 시점은 **2025년 9월**입니다. 이 시점을 기준으로 모든 추천이 이루어져야 합니다.
    사용자 정보를 바탕으로, **현재 신청 가능한** 지원사업 3개를 추천합니다.

    출력은 반드시 다음 규칙을 따르는 **하나의 JSON 객체**여야 합니다.
    - **JSON의 최상위 키는 반드시 "recommendations" 이어야 하며, 그 값은 3개의 추천 사업 객체를 담은 배열(Array)이어야 합니다.**

    각 추천 사업 객체는 '추천순위', '사업명', '설명', '주요조건', '신청정보' 키를 가져야 합니다.
    '주요조건'은 아래 예시처럼 신청기한, 연령, 거주, 소득 등 핵심 조건을 상세히 포함해야 합니다.
       (예시: "주요조건": {{ "신청기한": "2025년 9월 30일까지 (마감 임박)", "연령": "만 19세 ~ 39세", "거주": "신청일 기준 대전광역시에 1년 이상 계속 거주", "소득": "기준중위소득 150% 이하", "기타": "무주택자" }})
    '신청정보'는 아래 예시처럼 신청방법과 사용자의 직업을 고려한 필요서류를 포함해야 합니다.
       (예시: "신청정보": {{ "신청방법": "온라인 신청 (한국장학재단 홈페이지)", "필요서류": "신분증 사본, 주민등록등본, 재학증명서 (상세 서류는 공고 확인 필수)" }})

    JSON 외에 어떤 설명이나 텍스트도 추가해서는 안 됩니다.
    """
    user_prompt = f"아래 사용자 정보에 맞는 지원사업을 추천해주세요:\n{json.dumps(user_info, ensure_ascii=False)}"
    return system_prompt, user_prompt

def get_recommendations(user_info):
    system_prompt, user_prompt = create_recommendation_prompt(user_info)
    response = openai.chat.completions.create(
        model="gpt-4o",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )
    text_output = response.choices[0].message.content
    try:
        data = json.loads(text_output)
        return data.get("recommendations", [])
    except (json.JSONDecodeError, TypeError):
        print("추천 결과가 JSON 형식이 아닙니다:", text_output)
        return {"error": "추천 결과를 생성하지 못했습니다.", "details": text_output}

In [3]:
# 챗봇 + 추천 시스템 실행 예시

user_inputs = [
    "나는 대구 사는 남자 21살 학생이야. 소득은 없고 학점은 3.8 정도 돼.",
    "인천에 사는 35살 직장인입니다. 연봉은 4천만원 정도 됩니다."
]
for user_text in user_inputs:
    print("==================================================")
    print("[사용자 입력]", user_text)
    print("--------------------------------------------------")
    user_info = extract_user_info(user_text)
    print("[추출된 사용자 정보]")
    print(json.dumps(user_info, indent=2, ensure_ascii=False))
    print("--------------------------------------------------")
    recommendations = get_recommendations(user_info)
    print("[AI 추천 리스트 (3개 출력 보장)]")
    print(json.dumps(recommendations, indent=2, ensure_ascii=False))
    print("==================================================\n")

[사용자 입력] 나는 대구 사는 남자 21살 학생이야. 소득은 없고 학점은 3.8 정도 돼.
--------------------------------------------------
[추출된 사용자 정보]
{
  "성별": "남성",
  "지역": "대구",
  "소득": 0,
  "나이": 21,
  "직업": "학생"
}
--------------------------------------------------
[AI 추천 리스트 (3개 출력 보장)]
[
  {
    "추천순위": 1,
    "사업명": "대구 청년 월세 지원 사업",
    "설명": "대구 지역에 거주하는 청년들을 대상으로 월세를 지원하여 주거비 부담을 줄이는 사업입니다.",
    "주요조건": {
      "신청기한": "2025년 9월 30일까지",
      "연령": "만 19세 ~ 34세",
      "거주": "신청일 기준 대구광역시에 1년 이상 계속 거주",
      "소득": "기준중위소득 120% 이하",
      "기타": "무주택자"
    },
    "신청정보": {
      "신청방법": "온라인 신청 (대구시 복지포털)",
      "필요서류": "신분증 사본, 주민등록등본, 임대차계약서 사본"
    }
  },
  {
    "추천순위": 2,
    "사업명": "한국장학재단 국가장학금",
    "설명": "저소득층 대학생을 위한 등록금 지원 장학금으로, 학비 부담을 줄여줍니다.",
    "주요조건": {
      "신청기한": "2025년 10월 5일까지",
      "연령": "제한 없음",
      "거주": "대한민국 거주자",
      "소득": "기준중위소득 200% 이하",
      "기타": "대학 재학생"
    },
    "신청정보": {
      "신청방법": "온라인 신청 (한국장학재단 홈페이지)",
      "필요서류": "신분증 사본, 주민등록등본, 재학증명서, 소득증빙서류"
    }
  },